In [ ]:
from data_tools import get_data_with_dates, get_all_data
from tqdm.notebook import tqdm
from warnings import warn
import pandas as pd
from random import sample

In [ ]:
data = get_data_with_dates(get_all_data())

In [ ]:
samples = {}

for row in tqdm(data.itertuples(), total=len(data)):
    df = pd.read_excel(row.path)
    try:
        df = df[(df['Data wydania'] >= row.crisis_start - pd.Timedelta(days=30)) & (df['Data wydania'] < row.crisis_start + pd.Timedelta(days=30))]
        if df['Kryzys'].hasnans:
            if df['Kryzys'].nunique(dropna=False) != 2:
                warn(f'Invalid Kryzys column values in {row.name}.')
                continue
            df['label'] = ~df['Kryzys'].isna()
        else:
            df['Kryzys'] = df['Kryzys'].apply(lambda x: x[:3])
            if df['Kryzys'].nunique(dropna=False) != 2:
                warn(f'Invalid Kryzys column values in {row.name}.')
                continue
            df['label'] = df['Kryzys'] != 'NIE'
        sample_size = min(sum(df['label']), sum(~df['label']), 50)
        samples_neg = sample(df[~df['label']].index.to_list(), sample_size)
        samples_pos = sample(df[df['label']].index.to_list(), sample_size)
        samples[row.name] = samples_neg + samples_pos
        samples[row.name].sort()
    except KeyError:
        print(f'{row.name} has shit keys.')

In [ ]:
data.iloc[10]

In [ ]:
data